Tenemos los CSV, vamos a lanzar consultas en SQL, por lo que crearemos una Base de Datos, pasaremos los CSVs a 
tablas dentro de la Base de Datos y así podremos lanzar consultas

In [14]:
# Importar las bibliotecas necesarias
import pandas as pd  # Para manejar archivos CSV y bases de datos
import sqlite3  # Para interactuar con SQLite
import os  # Para comprobar la existencia de archivos

# 1. Conectar o crear la base de datos SQLite

In [15]:

# Si el archivo 'sales_data.db' no existe, SQLite lo creará automáticamente
conn = sqlite3.connect("sales_data.db")
print("Conexión establecida con la base de datos.")

Conexión establecida con la base de datos.


# 2. Definir los archivos CSV y los nombres de las tablas correspondientes y cargarlos a tablas

In [16]:

# Creamos un diccionario donde la clave es el nombre de la tabla en la base de datos
# y el valor es el nombre del archivo CSV
csv_files = {
    "customers": "customers.csv",
    "dim_customer": "dim_customer.csv",
    "dim_geography": "dim_geography.csv",
    "dim_product": "dim_product.csv",
    "dim_territory": "dim_territory.csv",
    "fact_sales": "fact_sales.csv"
}

# Cargar cada archivo CSV en su tabla correspondiente en SQLite
for table_name, file_name in csv_files.items():
    if os.path.exists(file_name):  # Verificamos que el archivo exista
        df = pd.read_csv(file_name)  # Leer el archivo CSV en un DataFrame
        df.to_sql(table_name, conn, if_exists="replace", index=False)  # Guardar en SQLite
        print(f"Tabla '{table_name}' cargada correctamente con {len(df)} registros.")
    else:
        print(f"El archivo '{file_name}' no se encontró. Verifica su nombre y ubicación.")

El archivo 'customers.csv' no se encontró. Verifica su nombre y ubicación.
Tabla 'dim_customer' cargada correctamente con 1360 registros.
Tabla 'dim_geography' cargada correctamente con 654 registros.
Tabla 'dim_product' cargada correctamente con 397 registros.
Tabla 'dim_territory' cargada correctamente con 11 registros.
Tabla 'fact_sales' cargada correctamente con 5954 registros.


# 4. Verificar la carga de datos mostrando la cantidad de registros en cada tabla

In [17]:
print("\nResumen de la carga de datos:")
for table_name in csv_files.keys():
    query = f"SELECT COUNT(*) FROM {table_name};"
    result = pd.read_sql_query(query, conn)
    print(f"La tabla '{table_name}' tiene {result.iloc[0, 0]} registros.")


Resumen de la carga de datos:
La tabla 'customers' tiene 1360 registros.
La tabla 'dim_customer' tiene 1360 registros.
La tabla 'dim_geography' tiene 654 registros.
La tabla 'dim_product' tiene 397 registros.
La tabla 'dim_territory' tiene 11 registros.
La tabla 'fact_sales' tiene 5954 registros.
